# Now for getters and setters!

## Motivation:
### Have a class Student that defines attributes that are themselves instances of a "TestScore" class. These attributes should only be overwritten if the TestScore.score is greater than the previous value. Each attribute of Student refers to a qualification he has, and each qualification is updated by a TestScore.


In [3]:
from typing import Optional


class PopQuiz:
    def __init__(self, subject: str, best_evaluation_score: int, test_day: str) -> None:
        self.subject: str = subject
        self.best_evaluation_score: int = best_evaluation_score
        self.test_day: str = test_day
    def __repr__(self):
        return f'{self.subject} score: {self.best_evaluation_score}'


class ReportCard:
    def __init__(
            self, 
            gym: Optional[PopQuiz] = None, 
            art: Optional[PopQuiz] = None, 
            math: Optional[PopQuiz] = None, 
            science: Optional[PopQuiz] = None, 
        ) -> None:
        self.gym: Optional[PopQuiz] = gym
        self.art: Optional[PopQuiz] = art
        self.math: Optional[PopQuiz] = math
        self.science: Optional[PopQuiz] = science


joe_grades = ReportCard()

joe_grades.math = PopQuiz("Math", 60, "Monday")
joe_grades.math = PopQuiz("Math", 50, "Monday")

joe_grades.math


Math score: 50

### This sucks cause Joe should have a 60 not a 50! Let's investigate how we can prevent assignment unless the score is better than the current score. But first, a look at getters and setters.

In [12]:
# A boring class with no properties
class A:
    def __repr__(self):
        return f'{vars(self)}'

a = A()

setattr(a, "some_attr", "some_value")
print(a.some_attr)
print(getattr(a, 'some_attr'))
print(getattr(a, 'non_existent_attr', "default_val_if_doesn't_exist"))

some_value
some_value
default_val_if_doesn't_exist


### Cool, cool. Let's go deeper. What if we want to add custom logic to the getters and setters? We can define them in the class like so... note that public attribute names are now actually METHODS with that attribute's name! And the 

### So unfortunately to avoid recusion error, we need to create two attrs for each instance attribute. Convention is to have the "private" one have an underscore

In [18]:
# A boring class with no properties
class A:
    def __repr__(self):
        return f'{vars(self)}'
    _x = 1
    @property
    def x(self):
        return self._x

a = A()

print(a.x)
a.x = 3

1


AttributeError: property 'x' of 'A' object has no setter

### Let's add a setter

In [25]:
# A boring class with no properties
class A:
    def __repr__(self):
        return f'{vars(self)}'
    _x = 1
    @property
    def x(self):
        return self._x
    @x.setter
    def x(self, new_val):
        self._x = new_val
    

a = A()

print(a.x)
a.x = 3
print(a.x)
print(a._x)

1
3
3


In [29]:
# A boring class with no properties
class A:
    def __init__(self,a):
        self._a = a
    @property
    def a(self):
        return self._a
    @a.setter
    def a(self, new_val):
        self._a = new_val+1
    

a = A(66)

print(a.a)
a.a = 3
print(a.a)
print(a._a)

66
4
4


### Try and factor out the @property stuff OUTSIDE of the class definition... but first more info on how decorators work

In [ ]:
# Don't worry too much about what the decorator does. Just note that the decorator itself takes in args. So how does
# the class definition get passed to the decorator?
def add_attribute(attribute_name, attribute_value):
    def class_decorator(cls):
        # Save a reference to the original __init__
        original_init = cls.__init__

        # Make a new __init__ that adds an attribute
        def new_init(self, *args, **kwargs):
            # Call the original __init__
            original_init(self, *args, **kwargs)

            # Add the new attribute
            setattr(self, attribute_name, attribute_value)

        # Replace the original __init__ with the new one
        cls.__init__ = new_init

        return cls

    return class_decorator


@add_attribute('new_attr', 42)
class A:
    def __init__(self, a):
        self._someattribute = a


# Test it out
a = A(10)
print(a._someattribute)  # 10
print(a.new_attr)  # 42





# When you write this
@add_attribute('new_attr', 42)
class A:
    def __init__(self, a):
        self._someattribute = a
        
# it is the same as this

class A:
    def __init__(self, a):
        self._someattribute = a

A = add_attribute('new_attr', 42)(A)

### In other words, the add_attribute('new_attr', 42) call returns a class decorator (which is itself a function), and then that decorator is called with A as its argument.

### add_attribute('new_attr', 42) is called, which returns the class_decorator function.
1. The class_decorator function is called with `A` as its argument `(class_decorator(A))`, which modifies `A'`s `__init__` method and then returns `A`.
2. The modified `A` is then assigned back to `A`.
3. This is why, after the decorator is applied, instances of `A` have a `new_attr` attribute. The decorator modified the `A` class to add this attribute during initialization.

In [45]:
# A boring class with no properties
class A:
    def __init__(self,a):
        self.someattribute = a
    

a = A(1)
    
print(vars(A.__init__))
print(vars(a).items())
[(a,b) for a,b in vars(a).items()]

{}
dict_items([('someattribute', 1)])


[('someattribute', 1)]

### So there's no way to get instance attribute data by looking at the class definition :( 
### Which means we need to go even deeper... We need to make a decorator that fucks with the class's `__init__` method if we want to edit instance data

### Let's make one that will add a new attribute with an underscore 

In [56]:
def func_that_takes_in_args_to_modify_a_decorator_and_then_return_it(num_times_to_print):
    def actual_decorator_that_takes_in_class_definition_and_fucks_with_it(cls):
        old_class_init_method = cls.__init__
        def new_init_method_that_messes_with_instance_attrs(self, *args, **kwargs): # this signature is becuase __init__ methods all have this signature
            old_class_init_method(self, *args, **kwargs) # run old init, to get all old instance attrs set
            # now lets inspect the instance attrs, which are available because __init__ is always passed `self` which is the instace!
            for attribute_name_str, attribute_val in vars(self).items():
                print(f'{attribute_name_str=}, {attribute_val*num_times_to_print}')
        cls.__init__ = new_init_method_that_messes_with_instance_attrs
    return actual_decorator_that_takes_in_class_definition_and_fucks_with_it

@func_that_takes_in_args_to_modify_a_decorator_and_then_return_it(num_times_to_print=3)
class A:
    def __init__(self,a):
        self.someattribute = a

print(type(A)) # prints <class 'NoneType'>

a = A(7) # Raises TypeError: 'NoneType' object is not callable

<class 'NoneType'>


TypeError: 'NoneType' object is not callable

In [60]:
# CORRECTED! Forgot to return the class from the decorator.
def func_that_takes_in_args_to_modify_a_decorator_and_then_return_it(num_times_to_print):
    def actual_decorator_that_takes_in_class_definition_and_fucks_with_it(cls):
        old_class_init_method = cls.__init__
        def new_init_method_that_messes_with_instance_attrs(self, *args, **kwargs): # this signature is becuase __init__ methods all have this signature
            old_class_init_method(self, *args, **kwargs) # run old init, to get all old instance attrs set
            # now lets inspect the instance attrs, which are available because __init__ is always passed `self` which is the instace!
            for attribute_name_str, attribute_val in vars(self).items():
                print(f'{attribute_name_str=}, {attribute_val*num_times_to_print}')
        cls.__init__ = new_init_method_that_messes_with_instance_attrs
        return cls
    return actual_decorator_that_takes_in_class_definition_and_fucks_with_it

@func_that_takes_in_args_to_modify_a_decorator_and_then_return_it(num_times_to_print=3)
class A:
    def __init__(self,a):
        self.someattribute = a

print(type(A)) # prints <class 'NoneType'>

a = A('&') # Raises TypeError: 'NoneType' object is not callable

<class 'type'>
attribute_name_str='someattribute', &&&


### Cool, cool.... now let's mess with it more. The end result doesn't need to parametrically edit the decorators, so let's just get rid of that shit anyway. Cool learning exercise though. It feels good to be back to only one nested function...


In [68]:
def actual_decorator_that_takes_in_class_definition_and_fucks_with_it(cls):
    old_class_init_method = cls.__init__
    def new_init_method_that_messes_with_instance_attrs(self, *args, **kwargs): 
        old_class_init_method(self, *args, **kwargs) # run old init, to get all old instance attrs set
        for attribute_name_str, attribute_val in vars(self).items():
            # add a new attribute that is the same name/val but with an underscore
            new_attr_name = f'_{attribute_name_str}'
            print(f'Duplicating {attribute_name_str} to {new_attr_name}')
            setattr(self, new_attr_name, attribute_val) #setattr(obj=self, name=new_attr_name, value=attribute_val) #no kwargs!
            print(f'Value of {new_attr_name} is {getattr(self,new_attr_name)}')
            
    cls.__init__ = new_init_method_that_messes_with_instance_attrs
    return cls

@actual_decorator_that_takes_in_class_definition_and_fucks_with_it
class A:
    def __init__(self,a):
        self.someattribute = a

print(vars(A),'\n')
a = A('&')
print('\n')
print(vars(a))
print('\n')

{'__module__': '__main__', '__init__': <function actual_decorator_that_takes_in_class_definition_and_fucks_with_it.<locals>.new_init_method_that_messes_with_instance_attrs at 0x11ec02700>, '__dict__': <attribute '__dict__' of 'A' objects>, '__weakref__': <attribute '__weakref__' of 'A' objects>, '__doc__': None} 

Duplicating someattribute to _someattribute
Value of _someattribute is &


RuntimeError: dictionary changed size during iteration

### Well, didn't know that was an error I could get. Guess I need to make a copy of it! Otherwise we're working directly with instance data. Need to create our own dict not tied to the same address


In [75]:
def actual_decorator_that_takes_in_class_definition_and_fucks_with_it(cls):
    old_class_init_method = cls.__init__
    def new_init_method_that_messes_with_instance_attrs(self, *args, **kwargs): 
        old_class_init_method(self, *args, **kwargs)
        old_instance_attrs_dict = vars(self).items().copy()
        for attribute_name_str, attribute_val in old_instance_attrs_dict:
            new_attr_name = f'_{attribute_name_str}'
            print(f'Duplicating {attribute_name_str} to {new_attr_name}')
            setattr(self, new_attr_name, attribute_val)
            print(f'Value of {new_attr_name} is {getattr(self,new_attr_name)}')
            
    cls.__init__ = new_init_method_that_messes_with_instance_attrs
    return cls

@actual_decorator_that_takes_in_class_definition_and_fucks_with_it
class A:
    def __init__(self,a):
        self.someattribute = a

print(vars(A),'\n')
a = A('&')
print('\n')
print(vars(a))
print('\n')

{'__module__': '__main__', '__init__': <function actual_decorator_that_takes_in_class_definition_and_fucks_with_it.<locals>.new_init_method_that_messes_with_instance_attrs at 0x11f0520c0>, '__dict__': <attribute '__dict__' of 'A' objects>, '__weakref__': <attribute '__weakref__' of 'A' objects>, '__doc__': None} 



AttributeError: 'dict_items' object has no attribute 'copy'

In [89]:
### But first some shenanigans with using dataclass

In [90]:
from dataclasses import dataclass

@dataclass
class V:
    a = 'attribute_name' # this makes it defined as a class attribute not an instance attribute 
    # fucking dataclasses this is retarded. I obviously want this to be an instance attribute with a default value

v = V()
print(v.a) # prints "attribute_name"... but it's a class attr not instance
print(f'{vars(v)=}')# is blank!!! We're calling vars (which prints the __dict__ of an obj) of an instance, not of the class 
print(f'{vars(V)=}')# shows a = attribute_name... smh


attribute_name
vars(v)={}
vars(V)=mappingproxy({'__module__': '__main__', 'a': 'attribute_name', '__dict__': <attribute '__dict__' of 'V' objects>, '__weakref__': <attribute '__weakref__' of 'V' objects>, '__doc__': 'V()', '__dataclass_params__': _DataclassParams(init=True,repr=True,eq=True,order=False,unsafe_hash=False,frozen=False), '__dataclass_fields__': {}, '__init__': <function V.__init__ at 0x11f14da80>, '__repr__': <function V.__repr__ at 0x11f14d580>, '__eq__': <function V.__eq__ at 0x11f14c900>, '__hash__': None, '__match_args__': ()})


### To use dataclasses to define instance attrs, don't give a default value!

In [92]:
from dataclasses import dataclass

@dataclass
class V:
    a:int # this makes it defined as an instance attribute, but then you have to pass a default value to the constructor for it
    # fucking dataclasses this is retarded

v = V()# fails because need to pass `a` now 
print(v.a) 
print(f'{vars(v)=}')
print(f'{vars(V)=}')


TypeError: V.__init__() missing 1 required positional argument: 'a'

In [93]:
from dataclasses import dataclass

@dataclass
class V:
    a:int = 3 # this makes it defined as an instance attribute, but then you have to pass a default value to the constructor for it
    # fucking dataclasses this is retarded

v = V()# fails because need to pass `a` now 
print(v.a) 
print(f'{vars(v)=}')
print(f'{vars(V)=}')


3
vars(v)={'a': 3}
vars(V)=mappingproxy({'__module__': '__main__', '__annotations__': {'a': <class 'int'>}, 'a': 3, '__dict__': <attribute '__dict__' of 'V' objects>, '__weakref__': <attribute '__weakref__' of 'V' objects>, '__doc__': 'V(a: int = 3)', '__dataclass_params__': _DataclassParams(init=True,repr=True,eq=True,order=False,unsafe_hash=False,frozen=False), '__dataclass_fields__': {'a': Field(name='a',type=<class 'int'>,default=3,default_factory=<dataclasses._MISSING_TYPE object at 0x105675e90>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),kw_only=False,_field_type=_FIELD)}, '__init__': <function V.__init__ at 0x11ec92980>, '__repr__': <function V.__repr__ at 0x11ec91580>, '__eq__': <function V.__eq__ at 0x11ec902c0>, '__hash__': None, '__match_args__': ('a',)})


# When using dataclass, Type hints imply instance attribute, lack of type hints implies class attribute

In [ ]:
# Back to the task at hand...

In [99]:
def actual_decorator_that_takes_in_class_definition_and_fucks_with_it(cls):
    old_class_init_method = cls.__init__
    def new_init_method_that_messes_with_instance_attrs(self, *args, **kwargs): 
        old_class_init_method(self, *args, **kwargs)
        old_instance_attrs_dict = {k:v for (k,v) in vars(self).items()}
        for attribute_name_str, attribute_val in old_instance_attrs_dict.items():
            new_attr_name = f'_{attribute_name_str}'
            print(f'Duplicating {attribute_name_str} to {new_attr_name}')
            setattr(self, new_attr_name, attribute_val)
            print(f'Value of {new_attr_name} is {getattr(self,new_attr_name)}')
            
    cls.__init__ = new_init_method_that_messes_with_instance_attrs
    return cls

@actual_decorator_that_takes_in_class_definition_and_fucks_with_it
class A:
    def __init__(self,a):
        self.someattribute = a

print(vars(A),'\n')
# prints lots of dunders and stuff
a = A('&')
# creating this instance will print "Duplicating ..." and "valoe of ... is ..."
print('\n')
print(vars(a))
print('\n')

{'__module__': '__main__', '__init__': <function actual_decorator_that_takes_in_class_definition_and_fucks_with_it.<locals>.new_init_method_that_messes_with_instance_attrs at 0x11ec02160>, '__dict__': <attribute '__dict__' of 'A' objects>, '__weakref__': <attribute '__weakref__' of 'A' objects>, '__doc__': None} 

Duplicating someattribute to _someattribute
Value of _someattribute is &


{'someattribute': '&', '_someattribute': '&'}


